In [1]:
import os
import sys
import json
import shutil
from pathlib import Path
from tempfile import TemporaryDirectory
from multiprocessing import Pool
import pandas as pd

In [2]:
sys.path.append('../../src')

from run import run_mlst, run_amrfinder, run_plasmidfinder, run_virulencefinder
from application import ribosomal_mlst
from utils import run_cmd

In [3]:
def run_lissero(infile, outfile):
    run_cmd(f'conda run -n lissero lissero {infile} > {outfile}')

In [4]:
def pipeline(infile, outdir, threads=4):
    os.makedirs(outdir, exist_ok=True)
    mlst_dirname = os.path.join(outdir, 'mlst')
    plasmidfinder_dirname = os.path.join(outdir, 'plasmidfinder')
#     virulencefinder_dirname = os.path.join(outdir, 'virulencefinder')
    amrfinder_filename = os.path.join(outdir, 'amrfinder.txt')
#     amrfinder_nucl_filename = os.path.join(outdir, 'amrfinder.fna')
    lissero_filename = os.path.join(outdir, 'lissero.txt')
#     rmlst_filename = os.path.join(outdir, 'rmlst.json')
    run_amrfinder(
        infile,
        amrfinder_filename,
        database='/media/GenomicResearch/Tools/amrfinder_database/latest/',
        threads=threads,
#         nucleotide_output=amrfinder_nucl_filename,
    )
    run_mlst(
        infile, mlst_dirname, '/media/GenomicResearch/Tools/CGE/mlst_db', 'lmonocytogenes'
    )
    run_plasmidfinder(
        infile, plasmidfinder_dirname, '/media/GenomicResearch/Tools/CGE/plasmidfinder_db'
    )
    run_lissero(
        infile, lissero_filename
    )
#     run_virulencefinder(
#         infile, virulencefinder_dirname, '/media/NAS/Central_Lab_Storage/bo-han/Data_Analysis/CGE/virulencefinder_db/', 'listeria'
#     )
#     ribosomal_mlst(
#         infile, rmlst_filename
#     )

In [5]:
dirpath = Path('/media/GenomicResearch/MiSeq/Listeria_monocytogenes/New')
outpath = Path('/media/GenomicResearch/MiSeq/Listeria_monocytogenes/Analysis')

In [6]:
with Pool(8) as p:
    try:
        for filepath in dirpath.iterdir():
            outdir = outpath/filepath.stem
            p.apply_async(pipeline, (filepath, outdir, 8))
        p.close()
        p.join()
    except:
        p.terminate()

In [5]:
pipeline('/media/GenomicResearch/MiSeq/Listeria_monocytogenes/New/R14.0396.fa',
         '/media/GenomicResearch/MiSeq/Listeria_monocytogenes/R14.0396', 32)